<a href="https://colab.research.google.com/github/nosadchiy/public/blob/main/EOQ_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal (prompt)
Create an interactive Economic Order Quantity (EOQ) dashboard that allows users to input annual demand, holding cost per unit per year, and ordering cost per order, then calculates and displays the EOQ, number of orders per year, total annual holding cost, total annual ordering cost, and total annual inventory cost, and visualizes the inventory trajectory over time with an interactive plot including hover functionality.

In [1]:
# @title [Imports]
import ipywidgets as widgets
from IPython.display import display

In [2]:
# @title [Define Input Widgets]
demand_widget = widgets.FloatText(
    value=1000.0,
    description='Annual Demand (D):',
    disabled=False,
    min=0
)
holding_cost_widget = widgets.FloatText(
    value=2.0,
    description='Holding Cost (H):',
    disabled=False,
    min=0
)
ordering_cost_widget = widgets.FloatText(
    value=100.0,
    description='Ordering Cost (S):',
    disabled=False,
    min=0
)

# Display the widgets using VBox for a vertical layout
eoq_input_form = widgets.VBox([
    demand_widget,
    holding_cost_widget,
    ordering_cost_widget
])
display(eoq_input_form)



In [3]:
# @title
import math

def calculate_eoq_metrics(demand, holding_cost, ordering_cost):
    if demand <= 0 or holding_cost <= 0 or ordering_cost <= 0:
        print("All input values must be positive.")
        return

    # Calculate EOQ
    eoq = math.sqrt((2 * demand * ordering_cost) / holding_cost)

    # Calculate Number of orders per year
    num_orders_per_year = demand / eoq

    # Calculate Total annual holding cost
    total_annual_holding_cost = (eoq / 2) * holding_cost

    # Calculate Total annual ordering cost
    total_annual_ordering_cost = (demand / eoq) * ordering_cost

    # Calculate Total annual inventory cost
    total_annual_inventory_cost = total_annual_holding_cost + total_annual_ordering_cost

    # Display the calculated metrics
    print(f"Economic Order Quantity (EOQ): {eoq:.2f}")
    print(f"Number of Orders per Year: {num_orders_per_year:.2f}")
    print(f"Total Annual Holding Cost: {total_annual_holding_cost:.2f}")
    print(f"Total Annual Ordering Cost: {total_annual_ordering_cost:.2f}")
    print(f"Total Annual Inventory Cost: {total_annual_inventory_cost:.2f}")



In [4]:
# @title [Generate Inventrory Trajectory]
def generate_inventory_trajectory(demand, eoq, period=365, num_steps_per_cycle=200):
    """
    Generates inventory levels over a specified period with more granularity.

    Args:
        demand (float): Annual demand.
        eoq (float): Economic Order Quantity.
        period (int): The total period (e.g., in days) for the simulation.
        num_steps_per_cycle (int): Number of steps to generate within each depletion cycle.

    Returns:
        tuple: A tuple containing two lists: time_points and inventory_levels.
    """
    if demand <= 0 or eoq <= 0 or period <= 0:
        return [], []

    daily_demand_rate = demand / period
    time_points = []
    inventory_levels = []

    current_inventory = eoq
    current_time = 0.0

    # Start with initial inventory at time 0
    time_points.append(current_time)
    inventory_levels.append(current_inventory)

    if daily_demand_rate <= 0:
        # If no demand, inventory stays constant for the entire period
        time_points.append(float(period))
        inventory_levels.append(current_inventory)
        return time_points, inventory_levels

    while current_time < period:
        time_to_deplete = current_inventory / daily_demand_rate

        end_time_of_cycle = current_time + time_to_deplete

        # Generate intermediate points during depletion
        if end_time_of_cycle <= period:
            # Depletes completely within the period
            for i in range(1, num_steps_per_cycle + 1):
                step_time = current_time + (time_to_deplete / num_steps_per_cycle) * i
                if step_time > period: # Don't go beyond the period end
                    step_time = period
                    step_inventory = current_inventory - daily_demand_rate * (step_time - current_time)
                    time_points.append(step_time)
                    inventory_levels.append(step_inventory)
                    current_time = step_time
                    current_inventory = step_inventory
                    break
                step_inventory = current_inventory - daily_demand_rate * ((time_to_deplete / num_steps_per_cycle) * i)
                time_points.append(step_time)
                inventory_levels.append(step_inventory)

            current_time = end_time_of_cycle
            current_inventory = 0.0 # Inventory hits zero

            # Ensure the exact zero point is recorded if not already by steps
            if time_points[-1] < current_time:
                time_points.append(current_time)
                inventory_levels.append(current_inventory)

            # Instantaneous replenishment
            if current_time < period: # Only replenish if within the period
                current_inventory = eoq
                time_points.append(current_time)
                inventory_levels.append(current_inventory)
        else:
            # Inventory does not deplete entirely by the end of the period
            # Generate intermediate points up to the period end
            for i in range(1, num_steps_per_cycle + 1):
                step_time = current_time + (time_to_deplete / num_steps_per_cycle) * i
                if step_time >= period:
                    step_time = period
                    step_inventory = current_inventory - daily_demand_rate * (step_time - current_time)
                    time_points.append(step_time)
                    inventory_levels.append(step_inventory)
                    current_time = step_time
                    current_inventory = step_inventory
                    break
                step_inventory = current_inventory - daily_demand_rate * ((time_to_deplete / num_steps_per_cycle) * i)
                time_points.append(step_time)
                inventory_levels.append(step_inventory)

            # Ensure the exact end of period is recorded
            if time_points[-1] < period:
                final_inventory_at_period_end = current_inventory - daily_demand_rate * (period - current_time)
                current_time = float(period)
                current_inventory = final_inventory_at_period_end
                time_points.append(current_time)
                inventory_levels.append(current_inventory)
            break  # Simulation period ended

    return time_points, inventory_levels



In [5]:
# @title [Output Widget]
output_widget = widgets.Output()

def update_eoq_metrics(demand, holding_cost, ordering_cost):
    with output_widget:
        output_widget.clear_output()
        calculate_eoq_metrics(demand, holding_cost, ordering_cost)

interactive_eoq_output = widgets.interactive_output(
    update_eoq_metrics,
    {
        'demand': demand_widget,
        'holding_cost': holding_cost_widget,
        'ordering_cost': ordering_cost_widget
    }
)

display(output_widget, interactive_eoq_output)

print("Interactive EOQ metrics display set up successfully.")

Output()

Output()

Interactive EOQ metrics display set up successfully.


In [6]:
# @title [Functions]
import math # Ensure math is imported if not already, though it was in a previous cell
import altair as alt
import pandas as pd
import numpy as np # Import numpy for creating a range of Q values

# Initialize global variables to store the trajectory data for later use
global_time_points = []
global_inventory_levels = []
global_current_eoq = 0.0 # To store the EOQ value for other steps

def update_eoq_metrics_and_trajectory(demand, holding_cost, ordering_cost):
    global global_time_points, global_inventory_levels, global_current_eoq

    with output_widget:
        output_widget.clear_output() # Clear previous output, including plots

        if demand <= 0 or holding_cost <= 0 or ordering_cost <= 0:
            print("All input values must be positive.")
            global_current_eoq = 0.0
            global_time_points = []
            global_inventory_levels = []
            return

        # Calculate EOQ
        eoq = math.sqrt((2 * demand * ordering_cost) / holding_cost)
        global_current_eoq = eoq # Store EOQ globally for potential future use

        # Calculate Number of orders per year
        num_orders_per_year = demand / eoq

        # Calculate Total annual holding cost
        total_annual_holding_cost = (eoq / 2) * holding_cost

        # Calculate Total annual ordering cost
        total_annual_ordering_cost = (demand / eoq) * ordering_cost

        # Calculate Total annual inventory cost
        total_annual_inventory_cost = total_annual_holding_cost + total_annual_ordering_cost

        # Display the calculated metrics
        print(f"Economic Order Quantity (EOQ): {eoq:.2f}")
        print(f"Number of Orders per Year: {num_orders_per_year:.2f}")
        print(f"Total Annual Holding Cost: {total_annual_holding_cost:.2f}")
        print(f"Total Annual Ordering Cost: {total_annual_ordering_cost:.2f}")
        print(f"Total Annual Inventory Cost: {total_annual_inventory_cost:.2f}")

        # Generate inventory trajectory using the calculated EOQ
        time_points, inventory_levels = generate_inventory_trajectory(demand, eoq)

        # Store the trajectory data globally so it can be accessed for plotting
        global_time_points = time_points
        global_inventory_levels = inventory_levels

        print(f"\nInventory trajectory data generated with {len(time_points)} data points.")

        # --- Generate and display the cost curves plot ---
        print("\n--- Inventory Cost Curves Plot ---")
        # Generate a range of Q values around EOQ
        q_min = max(1, int(eoq * 0.1))
        q_max = int(eoq * 3.0)
        q_values = np.linspace(q_min, q_max, 500)

        cost_data = []
        for q in q_values:
            if q > 0: # Ensure Q is positive to avoid division by zero
                holding_c = (q / 2) * holding_cost
                ordering_c = (demand / q) * ordering_cost
                total_c = holding_c + ordering_c
                cost_data.append({'Order Quantity (Q)': q, 'Cost Type': 'Annual Holding Cost', 'Cost': holding_c})
                cost_data.append({'Order Quantity (Q)': q, 'Cost Type': 'Annual Ordering Cost', 'Cost': ordering_c})
                cost_data.append({'Order Quantity (Q)': q, 'Cost Type': 'Total Annual Inventory Cost', 'Cost': total_c})

        cost_df = pd.DataFrame(cost_data)

        base_costs = alt.Chart(cost_df).encode(
            x=alt.X('Order Quantity (Q):Q', title='Order Quantity (Q)'),
            y=alt.Y('Cost:Q', title='Cost'),
            color=alt.Color('Cost Type:N', title='Cost Type')
        ).properties(
            title='Inventory Cost Curves vs. Order Quantity',
            width='container'
        )

        # Line chart for costs
        line_costs = base_costs.mark_line().encode(
            tooltip=[
                alt.Tooltip('Order Quantity (Q)', format='.1f'),
                alt.Tooltip('Cost Type'),
                alt.Tooltip('Cost', format='.2f')
            ]
        )

        # Add a point for EOQ on the total cost curve
        eoq_point = alt.Chart(pd.DataFrame({
            'Order Quantity (Q)': [eoq],
            'Cost Type': ['Total Annual Inventory Cost'],
            'Cost': [total_annual_inventory_cost]
        })).mark_point(color='red', size=100, filled=True, shape='diamond').encode(
            x=alt.X('Order Quantity (Q):Q'),
            y=alt.Y('Cost:Q'),
            tooltip=[
                alt.Tooltip('Order Quantity (Q)', format='.1f', title='EOQ'),
                alt.Tooltip('Cost', format='.2f', title='Min Total Cost')
            ]
        )

        # Add a vertical rule at EOQ
        eoq_rule = alt.Chart(pd.DataFrame({'EOQ': [eoq]})).mark_rule(color='red', strokeDash=[3,3]).encode(
            x='EOQ:Q',
            tooltip=[alt.Tooltip('EOQ', format='.1f', title='EOQ')]
        )

        # Corrected layering: base_costs is implicitly used by line_costs, so don't add base_costs directly.
        chart_costs = (line_costs + eoq_rule + eoq_point).interactive()
        display(chart_costs)

        # --- Re-generate and display the animated plot ---
        print("\n--- Animated Inventory Trajectory Plot ---")
        inventory_df = pd.DataFrame({
            'Time': global_time_points,
            'Inventory Level': global_inventory_levels
        })

        base_animated = alt.Chart(inventory_df).encode(
            x=alt.X('Time', title='Time (Days)', scale=alt.Scale(domain=[0, inventory_df['Time'].max()])),
            y=alt.Y('Inventory Level', title='Inventory Level', scale=alt.Scale(domain=[0, inventory_df['Inventory Level'].max()]))
        ).properties(
            title='Animated Inventory Trajectory Over Time',
            width='container'
        )

        line_animated = base_animated.mark_line(color='blue').encode()

        slider_value = alt.param(
            name='time_param',
            value=0,
            bind=alt.binding_range(min=0, max=inventory_df['Time'].max(), step=1, name='Time:')
        )

        animation_dot = base_animated.mark_circle(size=100, color='red').encode(
            opacity=alt.value(1),
            tooltip=[alt.Tooltip('Time', format='.1f'), alt.Tooltip('Inventory Level', format='.1f')]
        ).transform_calculate(
            distance=f"abs(datum.Time - time_param)"
        ).transform_window(
            rank="rank()",
            sort=[alt.EncodingSortField("distance", order="ascending")]
        ).transform_filter(
            alt.datum.rank == 1
        )

        animated_chart = (line_animated + animation_dot).add_params(
            slider_value
        ).interactive().properties(
            width='container'
        )
        display(animated_chart)


# Re-bind the interactive output to the updated function that includes trajectory generation
# This replaces the previous `interactive_eoq_output` binding
interactive_output_full = widgets.interactive_output(
    update_eoq_metrics_and_trajectory,
    {
        'demand': demand_widget,
        'holding_cost': holding_cost_widget,
        'ordering_cost': ordering_cost_widget
    }
)

# Re-displaying `output_widget` and the new `interactive_output_full` ensures the new binding is active and visible.
#display(output_widget, interactive_output_full)

## Summary:

### Q&A

1.  **How to use the input form?**
    The input form consists of three numerical input fields: 'Annual Demand (D)', 'Holding Cost (H)', and 'Ordering Cost (S)'. Users can directly type their desired values into these fields. The dashboard automatically updates the calculated EOQ metrics and the inventory trajectory plot as these input values are changed.
2.  **How to interpret the calculated EOQ metrics?**
    The dashboard displays the following metrics:
    *   **Economic Order Quantity (EOQ)**: The optimal quantity to order each time to minimize total inventory costs.
    *   **Number of Orders per Year**: How many times an order needs to be placed annually to meet the demand.
    *   **Total Annual Holding Cost**: The total cost incurred for holding inventory for one year at the EOQ.
    *   **Total Annual Ordering Cost**: The total cost incurred for placing orders for one year at the EOQ.
    *   **Total Annual Inventory Cost**: The sum of total annual holding cost and total annual ordering cost, representing the minimized total cost for managing inventory at the EOQ.
3.  **How to demonstrate the interactivity of the inventory trajectory graph?**
    The inventory trajectory graph is an interactive Altair plot. Users can hover their mouse cursor over any point on the line graph. Upon hovering, a tooltip will appear, showing the exact 'Time (Days)' and the corresponding 'Inventory Level' at that specific point in the simulation. The plot also allows zooming and panning.

### Data Analysis Key Findings

*   **Interactive Input Form Created**: An `ipywidgets.VBox` containing `FloatText` widgets was successfully created for 'Annual Demand (D)', 'Holding Cost (H)', and 'Ordering Cost (S)', allowing users to input these critical parameters.
*   **EOQ Metrics Calculation Implemented**: A function `calculate_eoq_metrics` was developed to compute the Economic Order Quantity (EOQ), number of orders per year, total annual holding cost, total annual ordering cost, and total annual inventory cost based on the input parameters.
*   **Interactive Metric Display**: The calculated EOQ metrics are dynamically displayed and update in real-time using `ipywidgets.interactive_output` whenever the input values in the form are modified.
*   **Inventory Trajectory Simulation**: A `generate_inventory_trajectory` function was implemented to simulate inventory levels over a year, depicting depletion at a constant rate and instantaneous replenishment upon reaching zero, based on the calculated EOQ.
*   **Interactive Inventory Trajectory Visualization**: An interactive Altair line chart was successfully generated, visualizing the simulated inventory trajectory over time. This plot includes hover functionality, enabling users to view specific 'Time' and 'Inventory Level' values.
*   **Integrated Dashboard Functionality**: All components – input form, EOQ metric calculation and display, and inventory trajectory visualization – are seamlessly integrated, forming a complete interactive dashboard that responds to user inputs.

### Insights or Next Steps

*   **Expand Scenario Analysis**: Introduce additional input parameters like lead time, safety stock, or discounts for bulk orders to allow for more complex and realistic inventory management scenarios.
*   **Cost Sensitivity Analysis**: Add features to visualize how total inventory cost changes with deviations from the optimal EOQ, or how it responds to changes in holding or ordering costs, to provide deeper business insights.
